# testing the modules

In [1]:
from pyapp.observation.phoneix import traced_agent,ph_instrumentor

# -------------------model -------------------#
from pyapp.model_connection.lm.langchain.litellm import get_model_mlflow_llamacpp, ModelConfig
### --- config --- ###
from pathlib import Path
from dotenv import load_dotenv
import os 
here = Path("./").resolve()
print(Path(here / "config.env").exists())
load_dotenv(here / "config.env")

mlflow_port = os.getenv("MLFLOW_PORT") or "5001"
tracking_uri=f"http://localhost:{mlflow_port}"
config_path=here / "model_dir"

### --- model --- ###
model_config = ModelConfig(model_name="rag_model",port=8080,gguf_relative_path="model_path/artifacts/model.gguf")
model = get_model_mlflow_llamacpp(tracking_uri,config_path,model_config , stream=True , mock_response="Hello world", temperature=0, max_tokens=20)



May 8, 2025 > 11:55:03 |  pyapp.observation.phoneix : 12 |   ℹ️ INFO  |  Starting Phoenix observation  
May 8, 2025 > 11:55:03 |  taskpy.main : 28 |   🐞 DEBUG  |  ['task', '--dir', PosixPath('/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/app/pyapp/src/pyapp/observation'), 'status']  
May 8, 2025 > 11:55:04 |  taskpy.main : 32 |   ❌ ERROR  |  task: [status] docker ps | grep lmorbits-obeservation | wc -l
  
May 8, 2025 > 11:55:04 |  taskpy.main : 34 |   ℹ️ INFO  |         1
  
May 8, 2025 > 11:55:04 |  pyapp.observation.instance : 34 |   ℹ️ INFO  |  Phoenix is already running  
True
May 8, 2025 > 11:55:04 |  pyapp.serve_integration.mlflow_llamacpp : 23 |   ℹ️ INFO  |  Config path: /Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/app/pyapp/notebooks/model_dir  
May 8, 2025 > 11:55:04 |  pyapp.serve_integration.mlflow_llamacpp : 24 |   ℹ️ INFO  |  MLflow client: <mlflow.tracking.client.MlflowClient object at 0x1093475f0>  
Ma

In [2]:
# -------------------traced_agent -------------------#
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "{input}"),
])

chain = prompt | model

@traced_agent(name="app1")
def assistant(messages: list[dict], session_id: str):
    # now you only have to do your business logic
    return chain.invoke(messages)

@traced_agent(name="app2")
def assistant2(messages: list[dict], session_id: str):
    # now you only have to do your business logic
    return chain.invoke(messages)


@traced_agent(name="app3")
def assistant3(messages: list[dict], session_id: str):
    answer, trace_url = assistant(messages, session_id)
    answer2, trace_url2 = assistant2(messages, session_id)
    return chain.invoke(messages)

messages = [
  {"role": "user", "content": "hi! im bob"}
]
import uuid
session_id = str(uuid.uuid4())
answer, trace_url = assistant3(messages, session_id)


In [3]:
trace_url

'http://localhost:6006/projects/UHJvamVjdDoy/traces/0c6a338e98f14f38b55b7e4805bf3c89?selectedSpanNodeId=U3Bhbjoy'

In [6]:

from openinference.instrumentation import using_session
from openinference.semconv.trace import SpanAttributes
from opentelemetry import trace # wherever your context manager lives
from functools import wraps

import base64
def traced_agent(name: str, propagate_session: bool = True, tracer_name:str="lmorbits-trace"):
    """
    Decorator that wraps a function in a span named `name`,
    automatically sets SESSION_ID, INPUT_VALUE, and OUTPUT_VALUE,
    and (optionally) enters using_session(session_id) around the call.
    """
    tracer = trace.get_tracer(tracer_name)
    def decorator(fn):
        @wraps(fn)
        def wrapper(messages: list[dict], session_id: str, *args, **kwargs):
            # start the OpenTelemetry span
            with tracer.start_as_current_span(
                name=name,
                attributes={SpanAttributes.OPENINFERENCE_SPAN_KIND: "agent"}
            ) as span:
                # record session and input
                trace_id = span.get_span_context().trace_id
                span_id = span.get_span_context().index(0)
                text = f"Span:{span_id}"
                st_big = base64.b64encode(text.encode("utf-8")).decode("utf-8")
                trace_id_hex = format(trace_id, "032x")  # Converts to 32-character hex
                
                trace_url = f"{ph_instrumentor.project_url}/traces/{trace_id_hex}?selectedSpanNodeId={st_big}"
                
                span.set_attribute(SpanAttributes.SESSION_ID, session_id)
                last_msg = messages[-1].get("content")
                span.set_attribute(SpanAttributes.INPUT_VALUE, last_msg)

                # optionally propagate session into sub‐spans
                if propagate_session:
                    with using_session(session_id):
                        result = fn(messages, session_id, *args, **kwargs)
                else:
                    result = fn(messages, session_id, *args, **kwargs)

                # record the output
                # assume returned object has .content
                output = getattr(result, "content", result)
                span.set_attribute(SpanAttributes.OUTPUT_VALUE, output)

                return result,trace_url
        return wrapper
    return decorator

@traced_agent(name="app1")
def assistant(messages: list[dict], session_id: str):
    # now you only have to do your business logic
    return chain.invoke(messages)

@traced_agent(name="app2")
def assistant2(messages: list[dict], session_id: str):
    # now you only have to do your business logic
    return chain.invoke(messages)


@traced_agent(name="app3")
def assistant3(messages: list[dict], session_id: str):
    answer, trace_url = assistant(messages, session_id)
    answer2, trace_url2 = assistant2(messages, session_id)
    return chain.invoke(messages)

messages = [
  {"role": "user", "content": "hi! im bob"}
]
import uuid
session_id = str(uuid.uuid4())


answer, trace_url = assistant3(messages, session_id)

print(trace_url)

http://localhost:6006/projects/UHJvamVjdDoy/traces/8a35fee6d45b2f15802acc9ea53b157b?selectedSpanNodeId=U3Bhbjoy


In [5]:
import uuid
messages = [
  {"role": "user", "content": "hi! im bob"}
]
session_id = str(uuid.uuid4())
assistant2(messages, session_id)

AIMessage(content="[{'role': 'user', 'content': 'hi! im bob'}]\n\n```", additional_kwargs={}, response_metadata={'token_usage': Usage(completion_tokens=20, prompt_tokens=37, total_tokens=57, completion_tokens_details=None, prompt_tokens_details=None), 'model': 'openai/custom', 'finish_reason': 'length', 'model_name': 'openai/custom'}, id='run--db61f2ab-b8b2-478a-a49b-806a2ebee493-0', usage_metadata={'input_tokens': 37, 'output_tokens': 20, 'total_tokens': 57})

In [13]:
from pyapp.observation.phoneix import observation

In [3]:
from openinference.instrumentation.helpers import get_span_id
import uuid
from opentelemetry import trace
import base64
tracer = trace.get_tracer("ap")
session_id = str(uuid.uuid4())
with tracer.start_as_current_span("app_test") as span:
    trace_id = span.get_span_context().trace_id
    span_id = span.get_span_context().index(0)
    text = f"Span:{span_id}"
    st_big = base64.b64encode(text.encode("utf-8")).decode("utf-8")
    print(st_big)
    trace_id_hex = format(trace_id, "032x")  # Converts to 32-character hex
    print(span_id)
    print(f"{ph_instrumentor.project_url}/traces/{trace_id_hex}?selectedSpanNodeId={st_big}")
    span.set_attribute("session_id", session_id)
    span.set_attribute("input_value", "test")
    answer = "hi there"
    span.set_attribute("output_value", answer)
    

U3Bhbjoy
2
http://localhost:6006/projects/UHJvamVjdDoy/traces/ca96755380cff2806a7c11695eb119b6?selectedSpanNodeId=U3Bhbjoy


http://localhost:6006/projects/UHJvamVjdDoy/traces/27baaf9284f28b54c94f169a3d1cc2b1?selectedSpanNodeId=U3Bhbjoz

U3BhbjozNQ==

In [23]:
st = "U3BhbjozNQ==" 
num = "7683441182680581977"
format(int(num), "032x")

'00000000000000006aa1110c0a488759'

In [24]:
import base64

# --- decode the given st back to text ---
st = "U3BhbjozNQ=="
decoded = base64.b64decode(st).decode("utf-8")
print(decoded)  
# → Span:35


# --- now encode a (possibly new) num to the same format ---
num = "35"                  # the number you want to package
text = f"Span:{num}"        # prefix it with "Span:"
st_new = base64.b64encode(text.encode("utf-8")).decode("utf-8")
print(st_new)
# → U3BhbjozNQ==

Span:35
U3BhbjozNQ==


In [25]:
num = "7683441182680581977"
text = f"Span:{num}"
st_big = base64.b64encode(text.encode("utf-8")).decode("utf-8")
print(st_big)

U3Bhbjo3NjgzNDQxMTgyNjgwNTgxOTc3


In [ ]:
U3Bhbjo3NjgzNDQxMTgyNjgwNTgxOTc3

import requests

response = requests.get(
    f"http://localhost:6006/v1/projects/{a.project_name}",
    headers={"Accept":"*/*"},
)

data = response.json()

In [35]:
data["data"]["id"]

'UHJvamVjdDoy'

In [11]:
number = 10459408256634934551
s = number.to_bytes((number.bit_length() + 7) // 8, 'big')  # Convert int to bytes
import base64
encoded = base64.b64encode(s).decode()
print(encoded)  # 'U3Bhbjo0'

kSdIZKGDVRc=


In [17]:
a.project_name

# testing stuff

In [1]:
from pyapp.model_connection.lm.langchain.litellm import get_model_mlflow_llamacpp, ModelConfig
### --- config --- ###
from pathlib import Path
from dotenv import load_dotenv
import os 
here = Path("./").resolve()
print(Path(here / "config.env").exists())
load_dotenv(here / "config.env")

mlflow_port = os.getenv("MLFLOW_PORT") or "5001"
tracking_uri=f"http://localhost:{mlflow_port}"
config_path=here / "model_dir"

### --- model --- ###
model_config = ModelConfig(model_name="rag_model",port=8080,gguf_relative_path="model_path/artifacts/model.gguf")
model = get_model_mlflow_llamacpp(tracking_uri,config_path,model_config , stream=True , mock_response="Hello world", temperature=0, max_tokens=20)

True
May 7, 2025 > 17:03:09 |  pyapp.serve_integration.mlflow_llamacpp : 23 |   ℹ️ INFO  |  Config path: /Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/app/pyapp/notebooks/model_dir  
May 7, 2025 > 17:03:09 |  pyapp.serve_integration.mlflow_llamacpp : 24 |   ℹ️ INFO  |  MLflow client: <mlflow.tracking.client.MlflowClient object at 0x10925efc0>  
May 7, 2025 > 17:03:09 |  pyapp.serve_integration.mlflow_llamacpp : 26 |   ℹ️ INFO  |  Experiments: [<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1745927919661, experiment_id='0', last_update_time=1745927919661, lifecycle_stage='active', name='Default', tags={}>]  
May 7, 2025 > 17:03:09 |  serve.servers.llamacpp.serve : 54 |   ℹ️ INFO  |  Initialized LLaMA CPP server manager  
May 7, 2025 > 17:03:09 |  serve.utils.model_config : 66 |   ℹ️ INFO  |  Initialized model configuration manager with config path: /Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/app/p

In [6]:
from opentelemetry.trace import SpanKind
from pyapp.observation.phoneix import PhonexLangChainInstrumentor
observation = PhonexLangChainInstrumentor("lmorbits-phoenix","app2")
tracer = observation.get_tracer("__name__")



Attempting to instrument while already instrumented
Overriding of current TracerProvider is not allowed


In [9]:
from loguru import logger
from taskpy import TaskCLI
from pathlib import Path
tracer = None

class PhoenixObservation:
  @property
  def port(self):
    return 6006

  @property
  def grpc_port(self):
    return 4317

  def __init__(self):
    self.task = TaskCLI(Path(__file__).parent )
  
  
  def is_running(self):
      try:
        answer = self.task.run("status")
        return int(answer.stdout.strip())!= 0
      except Exception as e:
        logger.error(f"Error in phoenix observation: {e}")
        return False 

  def start(self):
    try:
      status = self.is_running()
      if status:
        logger.info("Phoenix is already running")
      else: 
        self.task.run("start", port=self.port, grpc_port=self.grpc_port)
    except Exception as e:
      logger.error(f"Error in phoenix observation: {e}")

  def stop(self):
    try:
      status = self.is_running()
      if not status:
        logger.info("Phoenix is not running")
      else:
        self.task.run("stop")
    except Exception as e:
      logger.error(f"Error in phoenix observation: {e}")
  
  def remove(self):
    try:
      self.task.run("remove")
    except Exception as e:
      logger.error(f"Error in phoenix observation: {e}")


class PhonexLangChainInstrumentor:
  tracer_provider = None

  def __init__(self, project_name: str= "lmorbits-phoenix" , app_name: str= "app1"):
    try:
      self.app_name = app_name
      self.project_name = project_name
      from opentelemetry import trace
      from openinference.instrumentation.langchain import LangChainInstrumentor
      from phoenix.otel import register

      self.tracer_provider = register(
        project_name=self.project_name,
        set_global_tracer_provider=False,
        verbose=False
      ) 
      LangChainInstrumentor().instrument(tracer_provider=self.tracer_provider)
      trace.set_tracer_provider(self.tracer_provider)
      

    except Exception as e:
      logger.error(f"Error in phoenix observation: {e}")
  
  def get_tracer(self , module_name:str):
    from opentelemetry import trace
    return trace.get_tracer(f"{self.app_name}.{module_name}")
    
  
  


Attempting to instrument while already instrumented


In [12]:
from openinference.instrumentation import using_metadata
metadata = {
    "key-1": "value_1",
    "key-2": "value_2",
}

@using_metadata(metadata)
def call_fn(*args, **kwargs):
    print("call_fn")

In [ ]:
from openinference.instrumentation import TraceConfig


https://docs.arize.com/phoenix/tracing/how-to-tracing/setup-tracing/setup-sessions

In [6]:

from openinference.instrumentation import using_attributes, using_user , using_metadata, using_session
from openinference.semconv.trace import SpanAttributes
import uuid
from opentelemetry import trace

tracer = trace.get_tracer("1")


async def app1(message:str, session_id:str):

    with tracer.start_as_current_span(f"app1",kind=SpanKind.INTERNAL) as app1_span:
        app1_span.set_attribute(SpanAttributes.SESSION_ID, session_id)
        app1_span.set_attribute(SpanAttributes.INPUT_VALUE, message)
        with using_session(session_id):
            answer = ""
            async for chunk in model.astream(message):
                answer += chunk.content
                print(chunk, end="|", flush=True)
    
        app1_span.set_attribute(SpanAttributes.OUTPUT_VALUE, answer)
    return answer

# tracer2 = trace.get_tracer("2")
# @tracer2.start_as_current_span(f"app2",kind=SpanKind.INTERNAL, attributes={SpanAttributes.OPENINFERENCE_SPAN_KIND: "agent"})
# async def app2(message:str, session_id:str):
#     current_span = trace.get_current_span()
#     current_span.set_attribute(SpanAttributes.SESSION_ID, session_id)
#     current_span.set_attribute(SpanAttributes.INPUT_VALUE, message)
#     with using_session(session_id):
#         answer = ""
#         async for chunk in model.astream(message):
#             answer += chunk
#             print(chunk, end="|", flush=True)
    
#     current_span.set_attribute(SpanAttributes.OUTPUT_VALUE, answer)
#     return answer

# prompt_template = "Please describe the weather forecast for {city} on {date}"
# prompt_template_variables = {"city": "Johannesburg", "date":"July 11"}
# prompt_template_version = "v1.0"
# with using_attributes(
#     session_id="my-session-id",
#     user_id="my-user-id",
#     metadata=metadata,
#     tags=tags,
#     prompt_template=prompt_template,
#     prompt_template_version=prompt_template_version,
#     prompt_template_variables=prompt_template_variables,
# ):
    

with tracer.start_as_current_span(f"test",kind=SpanKind.CLIENT,) as main_span:
    session_id = str(uuid.uuid4())
    await app1("hello", session_id)
# app2("i", session_id)



NameError: name 'SpanKind' is not defined

In [66]:
import uuid

import openai
from openinference.instrumentation import using_session
from openinference.semconv.trace import SpanAttributes
from opentelemetry import trace

session_id = str(uuid.uuid4())

tracer = trace.get_tracer(__name__)

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("user", "{input}"),
])

chain = prompt | model

@tracer.start_as_current_span(name="app1", attributes={SpanAttributes.OPENINFERENCE_SPAN_KIND: "agent"})
def assistant(
  messages: list[dict],
  session_id: str = str,
):
  current_span = trace.get_current_span()
  current_span.set_attribute(SpanAttributes.SESSION_ID, session_id)
  current_span.set_attribute(SpanAttributes.INPUT_VALUE, messages[-1].get('content'))

  # Propagate the session_id down to spans crated by the OpenAI instrumentation
  # This is not strictly necessary, but it helps to correlate the spans to the same session
  with using_session(session_id):
     response = chain.invoke(messages)
     

  current_span.set_attribute(SpanAttributes.OUTPUT_VALUE, response.content)
  return response

@tracer.start_as_current_span(name="app2", attributes={SpanAttributes.OPENINFERENCE_SPAN_KIND: "agent"})
def assistant2(
  messages: list[dict],
  session_id: str = str,
):
  current_span = trace.get_current_span()
  current_span.set_attribute(SpanAttributes.SESSION_ID, session_id)
  current_span.set_attribute(SpanAttributes.INPUT_VALUE, messages[-1].get('content'))

  # Propagate the session_id down to spans crated by the OpenAI instrumentation
  # This is not strictly necessary, but it helps to correlate the spans to the same session
  with using_session(session_id):
     r = chain.invoke(messages)
     

  current_span.set_attribute(SpanAttributes.OUTPUT_VALUE, r.content)
  return r


@tracer.start_as_current_span(name="app3", attributes={SpanAttributes.OPENINFERENCE_SPAN_KIND: "agent"})
def assistant3(
  messages: list[dict],
  session_id: str = str,
):
  current_span = trace.get_current_span()
  current_span.set_attribute(SpanAttributes.SESSION_ID, session_id)
  current_span.set_attribute(SpanAttributes.INPUT_VALUE, messages[-1].get('content'))

  with using_session(session_id):
    answer = assistant( messages, session_id)
    answer2 = assistant2(messages, session_id)

    r = chain.invoke(messages)


  current_span.set_attribute(SpanAttributes.OUTPUT_VALUE, r.content)
  return r



messages = [
  {"role": "user", "content": "hi! im bob"}
]
response = assistant3(
  messages,
  session_id=session_id,
)


I0000 00:00:1746632636.184881  568758 chttp2_transport.cc:1201] ipv6:%5B::1%5D:4317: Got goaway [11] err=UNAVAILABLE:GOAWAY received; Error code: 11; Debug Text: ping_timeout {grpc_status:14, http2_error:11, created_time:"2025-05-07T18:43:56.184875+03:00"}


In [16]:

from openinference.instrumentation import using_session
from openinference.semconv.trace import SpanAttributes
from opentelemetry import trace # wherever your context manager lives
from functools import wraps


def traced_agent(name: str, propagate_session: bool = True, tracer_name:str="lmorbits-trace"):
    """
    Decorator that wraps a function in a span named `name`,
    automatically sets SESSION_ID, INPUT_VALUE, and OUTPUT_VALUE,
    and (optionally) enters using_session(session_id) around the call.
    """
    tracer = trace.get_tracer(tracer_name)
    def decorator(fn):
        @wraps(fn)
        def wrapper(messages: list[dict], session_id: str, *args, **kwargs):
            # start the OpenTelemetry span
            with tracer.start_as_current_span(
                name=name,
                attributes={SpanAttributes.OPENINFERENCE_SPAN_KIND: "agent"}
            ) as span:
                # record session and input
                trace_id = span.get_span_context().trace_id
                span_id = span.get_span_context().span_id
                trace_id_hex = format(trace_id, "032x")  # Converts to 32-character hex
                trace_url = f"{ph_instrumentor.project_url}/traces/{trace_id_hex}?selectedSpanNodeId={span_id}"
                
                span.set_attribute(SpanAttributes.SESSION_ID, session_id)
                last_msg = messages[-1].get("content")
                span.set_attribute(SpanAttributes.INPUT_VALUE, last_msg)

                # optionally propagate session into sub‐spans
                if propagate_session:
                    with using_session(session_id):
                        result = fn(messages, session_id, *args, **kwargs)
                else:
                    result = fn(messages, session_id, *args, **kwargs)

                # record the output
                # assume returned object has .content
                output = getattr(result, "content", result)
                span.set_attribute(SpanAttributes.OUTPUT_VALUE, output)

                return result,trace_url
        return wrapper
    return decorator

@traced_agent(name="app1")
def assistant(messages: list[dict], session_id: str):
    # now you only have to do your business logic
    return chain.invoke(messages)

@traced_agent(name="app2")
def assistant2(messages: list[dict], session_id: str):
    # now you only have to do your business logic
    return chain.invoke(messages)


@traced_agent(name="app3")
def assistant3(messages: list[dict], session_id: str):
    answer, trace_url = assistant(messages, session_id)
    answer2, trace_url2 = assistant2(messages, session_id)
    return chain.invoke(messages)

messages = [
  {"role": "user", "content": "hi! im bob"}
]
session_id = str(uuid.uuid4())


answer, trace_url = assistant3(messages, session_id)

print(trace_url)

NameError: name 'chain' is not defined

In [75]:
assistant3(messages,)

TypeError: assistant3() missing 1 required positional argument: 'session_id'

In [68]:
# Example of a well-structured decorator with proper documentation and error handling
def decorator_template(arg1, arg2=None):
    def decorator(func):
        from functools import wraps
        @wraps(func)  # Preserves the metadata of the original function
        def wrapper(*args, **kwargs):
            try:
                # Pre-processing logic here
                print(f"Before calling {func.__name__}")
                
                # Call the original function
                result = func(*args, **kwargs)
                
                # Post-processing logic here
                print(f"After calling {func.__name__}")
                
                return result
                
            except Exception as e:
                # Error handling logic
                print(f"Error in {func.__name__}: {str(e)}")
                raise     
        return wrapper
    return decorator

# Example usage:
@decorator_template("example", arg2="test")
def sample_function(x):
    return x * 2

sample_function(1)


Before calling sample_function
After calling sample_function


2

In [46]:
messages = [
  {"role": "user", "content": "hi! im bob"}
]
answer = chain.invoke(messages)

In [48]:
answer.content

"[{'role': 'user', 'content': 'hi! im bob'}]\n\n```"

In [20]:
with tracer.start_as_current_span(f"test",kind=SpanKind.CLIENT,) as main_span: 
    main_span.add_event("test event")
    with tracer.start_as_current_span(f"app1",metadata=dict(app="app1")) as app1_span: 
        async for chunk in model.astream("just say hello"):
            print(chunk, end="|", flush=True)
    with tracer.start_as_current_span(f"app2",kind=SpanKind.INTERNAL) as app2_span: 
        async for chunk in model.astream("just say hello"):
            print(chunk, end="|", flush=True)

I0000 00:00:1746628828.377568  568758 chttp2_transport.cc:1201] ipv6:%5B::1%5D:4317: Got goaway [11] err=UNAVAILABLE:GOAWAY received; Error code: 11; Debug Text: ping_timeout {created_time:"2025-05-07T17:40:28.377561+03:00", http2_error:11, grpc_status:14}


TypeError: OITracer.start_as_current_span() got an unexpected keyword argument 'metadata'

In [ ]:
SpanKind.INTERNAL

In [4]:
from pyapp.observation.phoneix import PhoenixObservation

phoenix = PhoenixObservation()
answer = phoenix.start()


2025-05-07 16:35:04.252 | DEBUG    | taskpy.main:run:28 - ['task', '--dir', PosixPath('/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/app/pyapp/src/pyapp/observation'), 'status']
2025-05-07 16:35:04.296 | ERROR    | taskpy.main:run:32 - task: [status] docker ps | grep lmorbits-obeservation | wc -l

2025-05-07 16:35:04.296 | INFO     | taskpy.main:run:34 -        0

2025-05-07 16:35:04.297 | DEBUG    | taskpy.main:run:28 - ['task', '--dir', PosixPath('/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/app/pyapp/src/pyapp/observation'), 'start', 'PORT=6006', 'GRPC_PORT=4317']
2025-05-07 16:35:06.206 | ERROR    | taskpy.main:run:32 - task: [start] docker pull arizephoenix/phoenix:latest
task: [start] docker run -d --name lmorbits-obeservation -p 6006:6006 -p 4317:4317 arizephoenix/phoenix:latest

2025-05-07 16:35:06.207 | INFO     | taskpy.main:run:34 - latest: Pulling from arizephoenix/phoenix
Digest: sha256:3d1baca9d83f83e403

In [2]:
phoenix.stop()

2025-05-07 16:01:17.790 | DEBUG    | taskpy.main:run:28 - ['task', '--dir', PosixPath('/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/app/pyapp/src/pyapp/observation'), 'status']
2025-05-07 16:01:17.839 | ERROR    | taskpy.main:run:32 - task: [status] docker ps | grep lmorbits-obeservation | wc -l

2025-05-07 16:01:17.839 | INFO     | taskpy.main:run:34 -        0

2025-05-07 16:01:17.839 | INFO     | pyapp.observation.phoneix:stop:32 - Phoenix is not running


In [2]:
phoenix.remove()

2025-05-07 16:34:55.461 | DEBUG    | taskpy.main:run:28 - ['task', '--dir', PosixPath('/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/app_projects/app/pyapp/src/pyapp/observation'), 'remove']
2025-05-07 16:34:56.106 | ERROR    | taskpy.main:run:32 - task: [stop] docker stop lmorbits-obeservation
task: [remove] docker rm lmorbits-obeservation

2025-05-07 16:34:56.106 | INFO     | taskpy.main:run:34 - lmorbits-obeservation
lmorbits-obeservation



In [3]:

dir(answer)

['__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'args',
 'check_returncode',
 'returncode',
 'stderr',
 'stdout']

In [4]:
answer.stdout()

TypeError: 'str' object is not callable